# Project: an interpretable VAE to study Transcription Factor rewiring

## Introduction

Contact: Carl Herrmann, Daria Doncevic (Heidelberg University carl.herrmann@bioquant.uni-heidelberg.de daria.doncevic@bioquant.uni-heidelberg.de)

Variational Autoencoders are now widely applied to capture the complexity in single-cell omics datasets and can be used to replace complete workflows of data analysis consisting of normalization, clustering, celltype annotation and differential expression analysis. However, due to their non-linear nature, they lack interpretability. As you have learned in week 2 of the course, a subset of VAE models such as the VEGA model ([Seninge *et al* (2021)](https://www.nature.com/articles/s41467-021-26017-0)) can convey intrinsic interpretability by incorporating biological networks directly into their model structure. In VEGA, every node in the latent space corresponds to a biological entity such as a pathway or transcription factor (TF) and is then connected only to the genes in the reconstruction layer that are annotated to that biological entity, making the one-layered decoder sparse. Thus, the activations of the nodes in the latent space can be interpreted as pathway or TF activities. In this project, you are going to focus more on the weights of the decoder connections than on the latent space activities, and see how they differ for different genes for a given TF. We believe that this change in the weights between TF and target genes might indicate a rewiring of the TF in different biological condition.
You are also going to estimate uncertainties for the weights (which also depends on the annotation that is used) and to investigate how the weights change in condition versus control, which we call "Transcription factor rewiring".

## Goal
The goal of this project is to study Transcription factor rewiring in the context of interferon response. Thus, you should focus especially on TFs that are known to play a role in interferon response, such as STAT1, STAT2, and the different IRFs, and their target genes. You can read more about that [in this review](https://www.nature.com/articles/nri3787). To be able to obtain different weights for condition and control, you will have to train two separate models. Since VAEs are probabilistic, you will end up with a different set of weights each time you train the model. Thus, your task will also be to estimate or compute uncertainties for the weights. There are different ways to achieve this:


*   Train each model multiple times (and then compute metrics such as mean and standard deviation for the different weights). Are there genes whose annotation to a given TF seems to be "less certain" than for others?
*   When we train multiple models and then aggregate their results, we also call this an ensemble. Training an ensemble is often expensive in terms of computational time and power. Stochastic Weight Averaging (SWA) has been proposed as a method that can yield more robust weights without the need of training ensembles. The publication is linked [here](https://arxiv.org/pdf/1803.05407), and additional information can be found [here](https://pytorch.org/blog/stochastic-weight-averaging-in-pytorch/). Try to implement this approach for your VEGA model.
*   Bayesian Neural Networks do built-in estimation of weight uncertainties. Try to replace your linear layer with a Bayesian layer to obtain these estimates. You can find some information to get you started [in this blogpost](https://towardsdatascience.com/from-theory-to-practice-with-bayesian-neural-network-using-python-9262b611b825/). More mathematical foundations can also be found [here](https://www.cs.toronto.edu/~duvenaud/distill_bayes_net/public/).

Do not forget to also include some biological speculation/interpretation of your results!

Questions

1) Which data we should choose for selecting weights -> processed data or raw regulon data -> if raw regulon data which column we should choose such as stimulation or inhibition colummn

2) In the mask, does 1 mean TF is regulating the gene and 0 means there is no regulation? What happens if we put -1 in the mask, as a way of incorporating inhibition regulation?

3) How to organize the final result? presentation? poster?

## Data and model

As in week 2, you will use a dataset that contains peripheral blood mononuclear cells (PBMCs) from systemic lupus patients, treated with Interferon beta or untreated (control). You can follow the steps from week 2's notebook to download the data. You will also work with a VEGA-like model, and you can reuse code from week 2 for the implementation.

This time, instead of Reactome pathways, you will use TF regulons from the collecTRI database as a biological prior. Information about this resource and how to obtain the data is available [here](https://github.com/saezlab/CollecTRI). You might want to filter the regulons based on regulon size prior to incorporation into the model.

# Load packages and dataset

In [ ]:
!pip install scanpy
!pip install omnipath
!pip install decoupler
!pip install torchbnn

## NEW PACKAGES
import os
import polars as pl   # install polars!!



import torch
import torch.nn as nn
import torch.utils
import torch.distributions
import torchvision
from torchvision import datasets, transforms
import math
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
from collections import OrderedDict, defaultdict
from collections import Counter

import decoupler as dc
import omnipath as op

device = 'cuda'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 2.1 MB/s eta 0:00:00
  Created wheel for docrep: filename=docrep-0.3.2-py3-none-any.whl size=19876 sha256=65a0be306c3ae032f360a62631d25f71c84001bdd494cb667dbcb697dafd6f9b
  Stored in directory: /root/.cache/pip/wheels/06/76/8f/0ecb7d357c0bff71a2bd1940671be2d07a200752da9189bb55
Successfully built docrep
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.6/122.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# download PBMC dataset for training
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zHJKoU8QcQB4cLR-oICO2YY4Nu-QaZHG' -O PBMC_train.h5ad
# load data as anndata object
PBMC_train = sc.read_h5ad("PBMC_train.h5ad")

# for validation
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rJKZYIG7rv7BQbDD9RElYOgHcxxzjcYj' -O PBMC_valid.h5ad
# load data as anndata object
PBMC_valid = sc.read_h5ad( "PBMC_valid.h5ad")

--2025-05-20 22:41:59--  https://docs.google.com/uc?export=download&id=1zHJKoU8QcQB4cLR-oICO2YY4Nu-QaZHG
Resolving docs.google.com (docs.google.com)... 192.178.163.102, 192.178.163.113, 192.178.163.139, ...
Connecting to docs.google.com (docs.google.com)|192.178.163.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1zHJKoU8QcQB4cLR-oICO2YY4Nu-QaZHG&export=download [following]
--2025-05-20 22:41:59--  https://drive.usercontent.google.com/download?id=1zHJKoU8QcQB4cLR-oICO2YY4Nu-QaZHG&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.199.132, 2607:f8b0:400e:c05::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.199.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45277554 (43M) [application/octet-stream]
Saving to: ‘PBMC_train.h5ad’

PBMC_train.h5ad     100%[===================>]  43.18M   183MB/s

NameError: name 'sc' is not defined

In [ ]:

# dc.op does not work for me
regulons = dc.get_collectri(organism='human', split_complexes=False)

filtered_regulons = regulons[regulons['source'].isin(
    [
      "GTF2I",
      "GTF3A",
      "NRF1",
      "ELF1",
      "STAT1",
      "STAT2",
      "IRF9",
      "STAT3",
      "STAT4",
      "STAT5A",
      "STAT5B",
      "IRF3",
      "IRF7",
      "IRF1",
      "IRF5",
     "IRF8",
     ])]

"""
['IRF1', 'IRF2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7','IRF8', 'IRF9', 'STAT1', 'STAT2', 'STAT3', 'STAT4', 'STAT5A','STAT5B', 'STAT6','GTF2I','RUNX1', 'MYOD1', 'SOX2','KLF4', 'THAP11','ADNP2', 'AEBP1','AHRR', 'ALX4','APBB1','APEX1', 'ARHGAP35','ARID1A']
"""

### TODO: NEED TO SET THRESHOLD FOR REGULON SIZE AND FILTER!!!!!!!

- median size?
- good to restrict the lower bound of regulon size because, essentially we are looking at how much genes are enriched in that TF or pathway. And, if we want a reliable proportion of weights to estimate how much each gene contribute in the pathway, we want enough number of genes regulated by that TF/pathway. So, filter out regulons smaller than 5.
- for the upper bound, unless the TF regulates like more than half of the genes in the dataset, it is fine to include them.


## Creating gmt files
Consider only stimulation or also train it for inhibition as well?
But this really depends because the direction of regulation depends on the tissues.

# VEGA Architecture

**VEGA model**
- a variational autoencoder model enhanced by gene annotations
- users can provide prior knowledge of gene to the *decoder* e.g. gene regulatory network
- allowing the latent variables to be directly interpretable

**Encoder**
- Two fully connected layer with dropout
- Since we are building a variational autoencoder which aims to learn a *probabilistic* latent space, we use a reparameterization trick.
 - Reparameterization trick is needed because backpropagation is computationally challenging when the latent space is probabilistic
 - So, we decompose the latent variable into $ z = \mu+\epsilon\cdot\sigma, \text{where } \epsilon \sim N(0,1)$. Then, gradients can be computed as the random variable $\epsilon$, independent of learnable parameters of the network ($\phi$ for encoder, $\theta$ for decoder).

**Decoder**
- to achieve biological interpretability, we must provide a gene module that defines how the nodes in decoder layers are connected
- this definition is fed to the model by creating a *mask*


**Dataset?**
- we use CollecTRI dataset to define transcriptional regulatory interactions (which TF targets which genes)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, latent_dims, input_dims, dropout, z_dropout): #dropout, z_dropout define the dropout rates of the encoder/latent space
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dims, 800),
                                     nn.ReLU(),
                                     nn.Dropout(p = dropout),
                                     nn.Linear(800, 800),
                                     nn.ReLU(),
                                     nn.Dropout(p = dropout))  #two layer, fully connected encoder with dropout


        self.mu = nn.Sequential(nn.Linear(800, latent_dims),
                                nn.Dropout(p = z_dropout))

        self.sigma = nn.Sequential(nn.Linear(800, latent_dims),
                                   nn.Dropout(p = z_dropout))

        self.N = torch.distributions.Normal(0, 1)     # epsilon(error)
        self.N.loc = self.N.loc.to(device)
        self.N.scale = self.N.scale.to(device)
        self.kl = 0

    def forward(self, x):
        x = self.encoder(x)
        mu =  self.mu(x)
        sigma = torch.exp(self.sigma(x)) # exp for numeric stability
        z = mu + sigma*self.N.sample(mu.shape)  # reparameterization trick
        self.kl = (0.5*sigma**2 + 0.5*mu**2 - torch.log(sigma) - 1/2).sum() #calculation of kullback-leibler divergence

        return z

In [ ]:

# rows should correspond to genes
# cols should correspond to gene sets/TFs
def create_mask(pbmc_train, pbmc_val, filtered_regulons, extra_nodes_count=1):
    '''
    filtered_regulons: collectri regulons filtered to contain only selected TFs
    pbmc_train: AnnData object
    pbmc_val: AnnData object
    extra_nodes_count: number of additional unannotated nodes to add to the mask

    Outputs:
    mask: torch tensor of shape (n_genes, n_TFs + extra_nodes_count)
    mask_df: polars DataFrame with the same shape as mask, containing the TFs as columns and genes as rows
    train: AnnData object with the same genes as mask_df
    valid: AnnData object with the same genes as mask_df
    '''
####FOR DTYPE COMPATIBILITY
    filtered_regulons = filtered_regulons.astype({
        "source": str,
        "target": str,
        "weight": np.float64
    })

    tmp = (
        pl
        .from_pandas(filtered_regulons)
        # alice and kerem and yusuf decided to convert all -1 to 1 and remove positive weights restriction
        .with_columns(
            pl.col("weight").replace(-1, 1)
        )
        .filter(
            pl.col("target").is_in(pbmc_train.var.index.to_numpy())
        )  # 901 x 13 -> 332 x 13
        .pivot(
            on="source",  # new columns
            index="target",  # stays in rows
            values="weight"
        )
        .fill_null(0)
    )

    for i in range(0, extra_nodes_count):
        node_index = i + 1
        node_name = "unannotated_" + str(node_index)
        tmp = tmp.with_columns(
            pl.lit(1).alias(node_name)
        )


    train = pbmc_train[:, tmp["target"].to_list()].copy()
    valid = pbmc_val[:, tmp["target"].to_list()].copy()

    return tmp, train, valid

In [ ]:
# define VEGA's decoder

class DecoderVEGA(nn.Module):
  """
  Define VEGA's decoder (sparse, one-layer, linear, positive)
  """
  def __init__(self,
               mask, soft_penalty):
        super(DecoderVEGA, self).__init__()

        self.sparse_layer = nn.Sequential(SparseLayer(mask, soft_penalty)) # we define the architecture of the decoder below with the class "SparseLayer"

  def forward(self, x):
    z = self.sparse_layer(x.to(device))
    return(z)

  def positive_weights(self):
      """
      constrain the decoder to positive weights (set negative weigths to zero)
      """
      w = self.sparse_layer[0].weight
      w.data = w.data.clamp(0)
      return w


# define a class SparseLayer, that specifies the decoder architecture (sparse connections based on the mask)
class SparseLayer(nn.Module):
  def __init__(self, mask, soft_penalty):
        """
        Extended torch.nn module which mask connection
        """
        super(SparseLayer, self).__init__()

        self.mask = nn.Parameter(torch.tensor(mask, dtype=torch.float).t(), requires_grad=False)
        self.weight = nn.Parameter(torch.Tensor(mask.shape[1], mask.shape[0]))
        self.bias = nn.Parameter(torch.Tensor(mask.shape[1]))
        self.soft_penalty = soft_penalty

        self.reset_parameters()

  def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

        # mask weight
        self.weight.data = self.weight.data * self.mask

  def forward(self, input):
        # See the autograd section for explanation of what happens here
        return SparseLayerFunction.apply(input, self.weight, self.bias, self.mask, self.soft_penalty)


######### You don't need to understand this part of the code in detail #########
class SparseLayerFunction(torch.autograd.Function):
    """
    We define our own autograd function which masks it's weights by 'mask'.
    For more details, see https://pytorch.org/docs/stable/notes/extending.html
    """

    # Note that both forward and backward are @staticmethods
    @staticmethod
    def forward(ctx, input, weight, bias, mask, soft_penalty):
        ctx.soft_penalty = soft_penalty
        weight = weight * (mask != 0) # change weight to 0 where mask == 0
        #calculate the output
        output = input.mm(weight.t())
        output += bias.unsqueeze(0).expand_as(output) # Add bias to all values in output
        ctx.save_for_backward(input, weight, bias, mask)
        return output

    @staticmethod
    def backward(ctx, grad_output): # define the gradient formula
        input, weight, bias, mask = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = grad_mask = None

        # These needs_input_grad checks are optional and only to improve efficiency
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
            # change grad_weight to 0 where mask == 0
            grad_weight = grad_weight * mask
        if ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias, grad_mask, None

In [ ]:
class VEGA(nn.Module):
    def __init__(self, latent_dims, input_dims, mask, dropout = 0.3, z_dropout = 0.3):
        super(VEGA, self).__init__()
        self.encoder = Encoder(latent_dims, input_dims, dropout, z_dropout) # we use the same encoder as before (two-layer, fully connected, non-linear)
        self.decoder = DecoderVEGA(mask, soft_penalty=0.1)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

# Training functions

In [ ]:
### training loop
def trainVEGA_with_valid(vae, data, val_data, epochs=100, beta = 0.0001, learning_rate = 0.001):
    opt = torch.optim.Adam(vae.parameters(), lr = learning_rate, weight_decay = 5e-4)
    train_losses = []
    valid_losses = []
    for epoch in range(epochs):
        train_loss_e = 0
        valid_loss_e = 0
        vae.train() #train mode

        for x in data:
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = vae(x)
            loss = ((x - x_hat)**2).sum() + beta* vae.encoder.kl
            loss.backward()
            opt.step()
            train_loss_e += loss.to('cpu').detach().numpy()
            vae.decoder.positive_weights() # we restrict the decoder to positive weights
        train_losses.append(train_loss_e/(len(data)*128))

        #### Here you should add the validation loop
        vae.eval()

        for x in val_data:
            x = x.to(device)
            x_hat = vae(x)
            loss = ((x - x_hat)**2).sum() + beta * vae.encoder.kl
            valid_loss_e += loss.to('cpu').detach().numpy()
        valid_losses.append(valid_loss_e/(len(val_data)*128))

        if epoch % 10 == 0:
            print("epoch: ", epoch, " train_loss: ", train_loss_e/(len(data)*128), "  valid_loss: ", valid_loss_e/(len(val_data)*128))

    return vae, train_losses, valid_losses

In [ ]:
def get_weight(vae):
  vae.eval()
  return vae.decoder.sparse_layer[0].weight.data.cpu().numpy()

# TODO: Implement Stochastic Weight Averaging (SWA)

* aggregates predictions of multiple models by SWA e.g.
  (cond1, cond2, ..) and (ctrl1, ctrl2, ..)
* can yield more robust weights
* SWA performs an equal average of the weights traversed by SGD with a modified learning rate schedule

In [ ]:
from torch.optim.swa_utils import AveragedModel, SWALR, update_bn

def trainVEGA_with_swa(vae, data, val_data, epochs=100, beta=0.0001,
                       learning_rate=0.001, swa_start=75, swa_lr=0.05):
    opt = torch.optim.Adam(vae.parameters(), lr=learning_rate, weight_decay=5e-4)
    swa_model = AveragedModel(vae)           # will hold the running average
    swa_scheduler = SWALR(opt, swa_lr=swa_lr)

    train_losses, valid_losses = [], []
    for epoch in range(epochs):
        vae.train()
        train_loss_e = 0.
        for x in data:
            x = x.to(device)
            opt.zero_grad()
            x_hat = vae(x)
            loss = ((x - x_hat)**2).sum() + beta * vae.encoder.kl
            loss.backward()
            opt.step()
            vae.decoder.positive_weights()
            train_loss_e += loss.detach().cpu().item()

        # start updating our SWA weights
        if epoch >= swa_start:
            swa_model.update_parameters(vae)
            swa_scheduler.step()

        train_losses.append(train_loss_e / (len(data) * data.batch_size))

        # validation
        vae.eval()
        valid_loss_e = 0.
        with torch.no_grad():
            for x in val_data:
                x = x.to(device)
                x_hat = vae(x)
                loss = ((x - x_hat)**2).sum() + beta * vae.encoder.kl
                valid_loss_e += loss.detach().cpu().item()
        valid_losses.append(valid_loss_e / (len(val_data) * val_data.batch_size))

        if epoch % 10 == 0:
            print(f"Epoch {epoch}: train_loss={train_losses[-1]:.4f}, valid_loss={valid_losses[-1]:.4f}")

    # recompute batch‐norm statistics for the SWA model (if you had any)
    update_bn(data, swa_model, device=device)

    return swa_model, train_losses, valid_losses


In [ ]:
def get_weight_swa(swa_model):
    """
    Extract the sparse‐decoder weights from an AveragedModel.
    """
    # if it's wrapped in AveragedModel, the averaged params live in swa_model.module
    model = swa_model.module if hasattr(swa_model, "module") else swa_model
    # pull out the sparse‐layer weight matrix
    W = model.decoder.sparse_layer[0].weight.data.cpu().numpy()
    return W

# TODO: Apply Bayesian layer

* replace nn.Linear in Encoder class with bnn.BayesLinear
* use nn.MSELoss() and bnn.BKLLoss() for optimization



In [ ]:
def _kl_loss(mu_0, log_sigma_0, mu_1, log_sigma_1) :
    """
    An method for calculating KL divergence between two Normal distribtuion.

    Arguments:
        mu_0 (Float) : mean of normal distribution.
        log_sigma_0 (Float): log(standard deviation of normal distribution).
        mu_1 (Float): mean of normal distribution.
        log_sigma_1 (Float): log(standard deviation of normal distribution).

    """
    kl = log_sigma_1 - log_sigma_0 + \
    (torch.exp(log_sigma_0)**2 + (mu_0-mu_1)**2)/(2*math.exp(log_sigma_1)**2) - 0.5
    return kl.sum()

In [ ]:
class BayesDecoder(nn.Module):
    def __init__(self, mask, soft_penalty):
        super(BayesDecoder, self).__init__()
        self.sparse_layer = nn.Sequential(BayesianSparseLayer(mask, soft_penalty))

    def forward(self, x):
        return self.sparse_layer(x.to(device))

    def kl_divergence(self):
        return self.sparse_layer[0].kl_loss()

    def positive_weights(self):
      """
      constrain the decoder to positive weights (set negative weigths to zero)
      """
      w = self.sparse_layer[0].weight_mu
      w.data = w.data.clamp(0)
      return w


class BayesianSparseLayer(nn.Module):
    def __init__(self, mask, soft_penalty):
        super(BayesianSparseLayer, self).__init__()

        self.mask = nn.Parameter(torch.tensor(mask, dtype=torch.float).t(), requires_grad=False)
        self.in_features = mask.shape[0]
        self.out_features = mask.shape[1]

        self.prior_mu = 0
        self.prior_sigma = 0.1
        self.prior_log_sigma = math.log(0.1)

        self.weight_mu = nn.Parameter(torch.Tensor(self.out_features, self.in_features))
        self.weight_log_sigma = nn.Parameter(torch.Tensor(self.out_features, self.in_features))

        self.bias_mu = nn.Parameter(torch.Tensor(self.out_features))
        self.bias_log_sigma = nn.Parameter(torch.Tensor(self.out_features))

        self.soft_penalty = soft_penalty

        self.reset_parameters()

    def reset_parameters(self):
        # Initialization method of Adv-BNN
        stdv = 1. / math.sqrt(self.weight_mu.size(1))
        self.weight_mu.data.uniform_(-stdv, stdv)
        self.weight_log_sigma.data.fill_(self.prior_log_sigma)
        self.bias_mu.data.uniform_(-stdv, stdv)
        self.bias_log_sigma.data.fill_(self.prior_log_sigma)
        # mask weight
        self.weight_mu.data = self.weight_mu.data * self.mask
        self.weight_log_sigma.data = self.weight_log_sigma * self.mask

    def forward(self, input):
        weight = self.weight_mu + torch.exp(self.weight_log_sigma) * torch.randn_like(self.weight_log_sigma)
        bias = self.bias_mu + torch.exp(self.bias_log_sigma) * torch.randn_like(self.bias_log_sigma)
        return SparseLayerFunction.apply(input, weight, bias, self.mask, self.soft_penalty)

    def kl_loss(self):
        return _kl_loss(self.weight_mu, self.weight_log_sigma,
                        self.prior_mu, self.prior_log_sigma)

class BayesVEGA(nn.Module):
    def __init__(self, latent_dims, input_dims, mask, dropout = 0.3, z_dropout = 0.3, soft_penalty = 0.1):
        super(BayesVEGA, self).__init__()
        self.encoder = Encoder(latent_dims, input_dims, dropout, z_dropout) # we use the same encoder as before (two-layer, fully connected, non-linear)
        self.decoder = BayesDecoder(mask, soft_penalty)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)


In [ ]:
def get_weight_bayes(model):
    # pull out the sparse‐layer weight matrix
    W = model.decoder.sparse_layer[0].weight_mu.data.cpu().numpy()
    return W

def get_weight_uncertainties_bayes(model):
    # pull out the sparse‐layer weight matrix
    W = model.decoder.sparse_layer[0].weight_log_sigma.data.cpu().numpy()
    return W

In [ ]:
### training loop
def trainVEGA_with_bayes(vae, data, val_data, epochs=100, beta_en = 0.0001, beta_de = 0.0001, learning_rate = 0.001):
    opt = torch.optim.Adam(vae.parameters(), lr = learning_rate, weight_decay = 5e-4)
    train_losses = []
    valid_losses = []
    for epoch in range(epochs):
        train_loss_e = 0
        valid_loss_e = 0
        vae.train() #train mode

        for x in data:
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = vae(x)
            kl_encoder = vae.encoder.kl
            kl_decoder = vae.decoder.kl_divergence()

            loss = ((x - x_hat)**2).sum() + beta_en* kl_encoder + beta_de*kl_decoder
            loss.backward()
            opt.step()
            train_loss_e += loss.to('cpu').detach().numpy()
            vae.decoder.positive_weights() # we restrict the decoder to positive weights
        train_losses.append(train_loss_e/(len(data)*128))

        #### Here you should add the validation loop
        vae.eval()

        for x in val_data:
            x = x.to(device)
            x_hat = vae(x)
            kl_encoder = vae.encoder.kl
            kl_decoder = vae.decoder.kl_divergence()
            loss = ((x - x_hat)**2).sum() + beta_en* kl_encoder + beta_de*kl_decoder
            valid_loss_e += loss.to('cpu').detach().numpy()
        valid_losses.append(valid_loss_e/(len(val_data)*128))

        if epoch % 10 == 0:
            print("epoch: ", epoch, " train_loss: ", train_loss_e/(len(data)*128), "  valid_loss: ", valid_loss_e/(len(val_data)*128))

    return vae, train_losses, valid_losses

# Split train/valid dataset
We need to train `PBMC_train_ctrl` and `PBMC_train_stim` separately.

In [ ]:
# Split data set into control and stimulated
PBMC_train_stim = PBMC_train[PBMC_train.obs["condition"] == "stimulated"]
print("PBMC_train_stim: ", len(PBMC_train_stim))
PBMC_train_ctrl = PBMC_train[PBMC_train.obs["condition"] == "control"]
print("PBMC_train_ctrl: ", len(PBMC_train_ctrl))
PBMC_valid_stim = PBMC_valid[PBMC_valid.obs["condition"] == "stimulated"]
print("PBMC_valid_stim: ", len(PBMC_valid_stim))
PBMC_valid_ctrl = PBMC_valid[PBMC_valid.obs["condition"] == "control"]
print("PBMC_valid_ctrl: ",len(PBMC_valid_ctrl))

PBMC_train_stim:  7109
PBMC_train_ctrl:  6406
PBMC_valid_stim:  1777
PBMC_valid_ctrl:  1601


# TODO: Train basic model (with activator or inhibitor gmt) on vega, swa, bayes
Train each model (cond/ctrl) multiple times (and then compute metrics such as mean and standard deviation for the different weights). Are there genes whose annotation to a given TF seems to be "less certain" than for others?

In [ ]:
def save_losses(train_losses, valid_losses, path, condition):
    os.makedirs(path, exist_ok=True)
    with open(os.path.join(path, f"{condition}, losses.csv"), "w") as f:
        f.write("epoch,train_loss,valid_loss\n")
        for epoch, (train_loss, valid_loss) in enumerate(zip(train_losses, valid_losses)):
            f.write(f"{epoch},{train_loss},{valid_loss}\n")

def run_vega_model(model_type, train_data, valid_data, path_to_save = 'models/vega', cond = 'all', cell_type = 'all', N=10, epochs = 60): # I need this encoder argument
    """
    cond is "stimulated" or "control" or 'all' (I need 'all' to train the encoder before frezeing it)
    model type is 'bayes', 'vega' or 'swa'
    """

    mask_df, train_data, valid_data = create_mask(train_data, valid_data, filtered_regulons)

    # convert mask polars to DF
    numeric_columns = [
    name for name, dtype in mask_df.schema.items()
    if dtype in (pl.Float32, pl.Float64, pl.Int32, pl.Int64)
    ]
    mask = mask_df.select(numeric_columns).to_numpy()
    mask = torch.from_numpy(mask).float()

    # create mask
    if not cond == 'all':
        train_data = train_data[train_data.obs["condition"] == cond]
        valid_data = valid_data[valid_data.obs["condition"] == cond]

    if not cell_type == 'all':
        train_data = train_data[train_data.obs["cell_type"] == cell_type]
        valid_data = valid_data[valid_data.obs["cell_type"] == cell_type]

    # set up dataloader
    trainX = torch.utils.data.DataLoader(train_data.X.toarray(), batch_size=128, shuffle=True)
    validX = torch.utils.data.DataLoader(valid_data.X.toarray(), batch_size=128, shuffle=True)

    all_weights = []

    match model_type:
        case "vega":
            for _ in range(N):
                os.makedirs(path_to_save, exist_ok=True)
                vega = VEGA(latent_dims= mask.shape[1], input_dims = mask.shape[0], mask = mask.T, dropout = 0.3, z_dropout = 0.3).to(device)    #   REPLACE THIS WITH YOUR VEGA MODEL
                vega, train_losses, valid_losses = trainVEGA_with_valid(vega, trainX, validX, epochs=epochs, beta=0.0001)
                weight = get_weight(vega)
                all_weights.append(weight)
                save_losses(train_losses, valid_losses, path_to_save, cond)  # there is no need to plot it, let's save a loss function to plot it later

        case "bayes":
            weight_uncertainties = []
            for _ in range(N):
                os.makedirs(path_to_save, exist_ok=True)
                vega = BayesVEGA(latent_dims= mask.shape[1], input_dims = mask.shape[0], mask = mask.T, dropout = 0.3,  z_dropout = 0.3).to(device)          #   REPLACE THIS WITH YOUR VEGA MODEL
                vega, train_losses, valid_losses = trainVEGA_with_bayes(vega, trainX, validX, epochs = epochs, beta_en = 0.0001, beta_de=0.0001)
                weight = get_weight_bayes(vega)
                uncertainty = get_weight_uncertainties_bayes(vega)
                uncertainty = np.exp(uncertainty)
                all_weights.append(weight)
                weight_uncertainties.append(uncertainty)
                save_losses(train_losses, valid_losses, path_to_save, cond)

            weight_uncertainties = np.stack(weight_uncertainties)
            mean_weight_uncertainties = weight_uncertainties.mean(axis=0)
            mean_weight_uncertainties = pd.DataFrame(mean_weight_uncertainties, index = train_data.var.index.tolist(), columns=mask_df.columns[1:])
            mean_weight_uncertainties.to_csv(os.path.join(path_to_save, f"{cond}_{model_type}_{cell_type}_weight_uncertainty_mean.csv"), index=True)

        case "swa":
            for _ in range(N):
                os.makedirs(path_to_save, exist_ok=True)
                vega = VEGA(latent_dims=mask.shape[1],
                        input_dims=mask.shape[0],
                        mask=mask.T,
                        dropout=0.3,
                        z_dropout=0.3).to(device)

                swa_model, train_losses, valid_losses = trainVEGA_with_swa(
                    vega, trainX, validX,
                    epochs=100, beta=0.0001,
                    learning_rate=0.0005,
                    swa_start=80,
                    swa_lr=0.0001
                )

                W_swa = get_weight_swa(swa_model)
                all_weights.append(W_swa)
                save_losses(train_losses, valid_losses, path_to_save, cond)


    stacked = np.stack(all_weights)
    mean_weight = stacked.mean(axis=0)
    std_weight = stacked.std(axis=0)

    mean_weight = pd.DataFrame(mean_weight, index = train_data.var.index.tolist(), columns=mask_df.columns[1:])
    std_weight = pd.DataFrame(std_weight, index = train_data.var.index.tolist(), columns=mask_df.columns[1:])


    mean_weight.to_csv(os.path.join(path_to_save, f"{cond}_{model_type}_{cell_type}_weight_mean.csv"), index=True)
    std_weight.to_csv(os.path.join(path_to_save, f"{cond}_{model_type}_{cell_type}_weight_std.csv"), index=True)

    return vega, mean_weight, std_weight

In [ ]:
vega, mean_weight, std_weight = run_vega_model("vega", PBMC_train_stim, PBMC_valid_stim, path_to_save = 'models/vega', cond = 'stimulated', cell_type = 'all', N=1, epochs = 60)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

def plot_tf_rewiring(path, tf_names, title,
    n_cols=3, figsize=(15, 12)
):
    for root, dirs, files in os.walk(path):
        for filename in files:
            print(f"Processing file: {filename}")
            if 'bayes' in filename:
                if filename.endswith('uncertainty_mean.csv'):
                    if 'control' in filename:
                        file_path = os.path.join(root, filename)
                        sigma_weights_ctrl = pd.read_csv(file_path, index_col=0)
                    elif 'stimulated' in filename:
                        file_path = os.path.join(root, filename)
                        sigma_weights_stim = pd.read_csv(file_path, index_col=0)
            else:
                if filename.endswith('std.csv'):
                    if 'control' in filename:
                        file_path = os.path.join(root, filename)
                        sigma_weights_ctrl = pd.read_csv(file_path, index_col=0)
                    elif 'stimulated' in filename:
                        file_path = os.path.join(root, filename)
                        sigma_weights_stim = pd.read_csv(file_path, index_col=0)

            if filename.endswith('weight_mean.csv'):
                if 'control' in filename:
                    file_path = os.path.join(root, filename)
                    mean_weights_ctrl = pd.read_csv(file_path, index_col=0)
                elif 'stimulated' in filename:
                    file_path = os.path.join(root, filename)
                    mean_weights_stim = pd.read_csv(file_path, index_col=0)

    n_plots = len(tf_names)
    n_rows = (n_plots + n_cols - 1) // n_cols

    fig, axs = plt.subplots(n_rows, n_cols, figsize=figsize, squeeze=False)
    fig.suptitle(title, fontsize=16)

    for idx, tf_name in enumerate(tf_names):
        row, col = divmod(idx, n_cols)
        ax = axs[row][col]

        # Filter TF target genes
        stim = mean_weights_stim[tf_name].rename("stim")
        ctrl = mean_weights_ctrl[tf_name].rename("ctrl")

        std_stim = sigma_weights_stim[tf_name].rename("std_stim")
        std_ctrl = sigma_weights_ctrl[tf_name].rename("std_ctrl")

        merged = pd.concat([ctrl, stim, std_ctrl, std_stim], axis=1).dropna()
        merged["uncertainty"] = merged[["std_ctrl","std_stim"]].mean(axis=1)

        ax.errorbar(
            merged["ctrl"], merged["stim"],
            xerr=merged["std_ctrl"], yerr=merged["std_stim"],
            fmt='.',                 # circle markers
            ecolor='black',           # color of error bars
            elinewidth=0.5,          # thin error bars
            markersize=5,
            markeredgecolor='blue',
            markeredgewidth=0.5
        )

        ax.plot([merged["ctrl"].min(), merged["ctrl"].max()],
        [merged["ctrl"].min(), merged["ctrl"].max()],
        color='grey', linestyle='--')
        ax.set_xlabel("Control mean weight")
        ax.set_ylabel("Stimulated mean weight")
        ax.set_title(f"{tf_name} weight: control vs. stimulated")

        merged["dist"] = np.abs(merged["stim"] - merged["ctrl"]) / np.sqrt(2)
        top = merged.nlargest(20, "dist")
        for gene, row in top.iterrows():
            ax.annotate(gene, (row["ctrl"], row["stim"]), fontsize=8)

        plt.tight_layout()
    plt.show()

plot_tf_rewiring(
    path="./model/bayes/",
    tf_names=["STAT1", "STAT2"],
    title="TF Rewiring: Control vs Stimulated",
)

# Train model with merged mask


In [ ]:
def create_merged_mask(adata, activator_gmt, inhibitor_gmt, add_nodes:int=1, sep='\t'):

    activator_dict = OrderedDict()
    if activator_gmt is not None:
        with open(activator_gmt) as f:
            for line in f.readlines():
                line = line.strip()
                spli = line.split(sep)
                activator_dict[spli[0]] = spli[2:]

    inhibitor_dict = OrderedDict()
    if inhibitor_gmt is not None:
        with open(inhibitor_gmt) as f:
            for line in f.readlines():
                line = line.strip()
                spli = line.split(sep)
                inhibitor_dict[spli[0]] = spli[2:]

    feature_list = adata.var.index.tolist()

    unique_keys = set(activator_dict.keys()) | set(inhibitor_dict.keys())

    # Create mask
    mask = np.zeros((len(feature_list), len(unique_keys)))
    for j, k in enumerate(activator_dict.keys()):
        for i in range(mask.shape[0]):
            if feature_list[i] in activator_dict[k]:      # match genes
                mask[i,j] = 1.      # 1 means target is stimulated by TF, 0 means target is not affected by TF

    for j, k in enumerate(inhibitor_dict.keys()):
        for i in range(mask.shape[0]):
            if feature_list[i] in inhibitor_dict[k]:      # match genes
                mask[i,j] = -1.

     # Add unannotated nodes
    vec = np.ones((mask.shape[0], add_nodes))
    mask = np.hstack((mask, vec))

    adata = adata.copy()
    adata.uns['_vega'] = dict() #create attribute "_vega" to store the mask and pathway information
    adata.uns['_vega']['mask'] = mask
    adata.uns['_vega']['gmv_names'] = list(unique_keys) + ['UNANNOTATED_'+str(k) for k in range(add_nodes)]

    return adata, mask

### 1. Basic model + merged mask

In [ ]:
class MergedDecoderVEGA(nn.Module):
    """
    Define VEGA's decoder (sparse, one-layer, linear, positive)
    """
    def __init__(self, mask, soft_penalty):
        super(MergedDecoderVEGA, self).__init__()

        self.sparse_layer = nn.Sequential(MergedSparseLayer(mask, soft_penalty)) # we define the architecture of the decoder below with the class "SparseLayer"

    def forward(self, x):
        z = self.sparse_layer(x.to(device))
        return(z)

# define a class SparseLayer, that specifies the decoder architecture (sparse connections based on the mask)
class MergedSparseLayer(SparseLayer):
    def __init__(self, mask, soft_penalty):
        """
        Extended torch.nn module which mask connection
        """
        super().__init__(mask, soft_penalty)

        self.activator_mask = (self.mask == 1).float()
        self.inhibitor_mask = (self.mask == -1).float()
        self.zero_mask = (self.mask == 0).float()
        self.soft_penalty = self.soft_penalty

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    def constrain_weights(self):
        with torch.no_grad():
            # Clamp activator weights to be >= 0
            device = self.weight.device

            a_mask = self.activator_mask.to(device)
            i_mask = self.inhibitor_mask.to(device)
            z_mask = self.zero_mask.to(device)

            self.weight.data = self.weight.data * (1 - a_mask) + \
                               self.weight.data.clamp(min=0) * a_mask

            # Clamp inhibitor weights to be <= 0
            self.weight.data = self.weight.data * (1 - i_mask) + \
                               self.weight.data.clamp(max=0) * i_mask

            # Set zero-regulation weights to exactly zero
            self.weight.data = self.weight.data * (1 - z_mask)

    def forward(self, input):
        # See the autograd section for explanation of what happens here
        return MergedSparseLayerFunction.apply(input, self.weight, self.bias, self.mask, self.soft_penalty)


######### You don't need to understand this part of the code in detail #########
class MergedSparseLayerFunction(torch.autograd.Function):
    """
    We define our own autograd function which masks it's weights by 'mask'.
    For more details, see https://pytorch.org/docs/stable/notes/extending.html
    """

    # Note that both forward and backward are @staticmethods
    @staticmethod
    def forward(ctx, input, weight, bias, mask, soft_penalty=0.1):
        ctx.soft_penalty = soft_penalty
        return SparseLayerFunction.forward(ctx, input, weight, bias, mask, soft_penalty)

    @staticmethod
    def backward(ctx, grad_output): # define the gradient formula
        input, weight, bias, mask = ctx.saved_tensors
        soft_penalty = ctx.soft_penalty
        grad_input = grad_weight = grad_bias = grad_mask = None

        # These needs_input_grad checks are optional and only to improve efficiency
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight * (mask != 0))
        if ctx.needs_input_grad[1]:
            raw_grad_weight = grad_output.t().mm(input)

            grad_weight = torch.zeros_like(raw_grad_weight)

            # Activators (mask = 1): allow positive weights, penalize negatives
            activator_mask = (mask == 1)
            grad_weight += activator_mask * raw_grad_weight
            grad_weight -= activator_mask * (weight < 0).float() * weight.abs() * soft_penalty  # soft penalty

            # Inhibitors (mask = -1): allow negative weights, penalize positives
            inhibitor_mask = (mask == -1)
            grad_weight += inhibitor_mask * raw_grad_weight
            grad_weight += inhibitor_mask * (weight > 0).float() * weight.abs() * soft_penalty  # soft penalty

            # Zeros: do not update
            grad_weight *= (mask != 0).float()

        if ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias, None, None

In [ ]:
class MergedVEGA(nn.Module):
    def __init__(self, latent_dims, input_dims, mask, dropout = 0.3, z_dropout = 0.3, soft_penalty = 0.1):
        super(MergedVEGA, self).__init__()
        self.encoder = Encoder(latent_dims, input_dims, dropout, z_dropout) # we use the same encoder as before (two-layer, fully connected, non-linear)
        self.decoder = MergedDecoderVEGA(mask, soft_penalty)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [ ]:
### training loop
def trainMergedVEGA_with_valid(vae, data, val_data, epochs=100, beta = 0.0001, learning_rate = 0.001):
    opt = torch.optim.Adam(vae.parameters(), lr = learning_rate, weight_decay = 5e-4)
    train_losses = []
    valid_losses = []
    for epoch in range(epochs):
        train_loss_e = 0
        valid_loss_e = 0
        vae.train() #train mode

        for x in data:
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = vae(x)
            loss = ((x - x_hat)**2).sum() + beta* vae.encoder.kl
            loss.backward()
            opt.step()
            train_loss_e += loss.to('cpu').detach().numpy()
            vae.decoder.sparse_layer[0].constrain_weights() # we restrict the decoder to positive weights
        train_losses.append(train_loss_e/(len(data)*128))

        #### Here you should add the validation loop
        vae.eval()

        for x in val_data:
            x = x.to(device)
            x_hat = vae(x)
            loss = ((x - x_hat)**2).sum() + beta * vae.encoder.kl
            valid_loss_e += loss.to('cpu').detach().numpy()
        valid_losses.append(valid_loss_e/(len(val_data)*128))

        if epoch % 10 == 0:
            print("epoch: ", epoch, " train_loss: ", train_loss_e/(len(data)*128), "  valid_loss: ", valid_loss_e/(len(val_data)*128))

    return vae, train_losses, valid_losses



In [ ]:
merged_vega_stim, merged_mean_weight_stim, merged_std_weight_stim = run_merged_vega_model(PBMC_train_stim, PBMC_valid_stim, "stimulated", "activator.gmt", "inhibitor.gmt", 1)

In [ ]:
merged_vega_ctrl, merged_mean_weight_ctrl, merged_std_weight_ctrl = run_merged_vega_model(PBMC_train_ctrl, PBMC_valid_ctrl, "control", "activator.gmt", "inhibitor.gmt", 1)

### 2. SWA model + merged mask

In [ ]:
def trainMergedVEGA_with_swa(vae, data, val_data, epochs=100, beta=0.0001,
                       learning_rate=0.001, swa_start=75, swa_lr=0.05):
    opt = torch.optim.Adam(vae.parameters(), lr=learning_rate, weight_decay=5e-4)
    swa_model = AveragedModel(vae)           # will hold the running average
    swa_scheduler = SWALR(opt, swa_lr=swa_lr)

    train_losses, valid_losses = [], []
    for epoch in range(epochs):
        vae.train()
        train_loss_e = 0.
        for x in data:
            x = x.to(device)
            opt.zero_grad()
            x_hat = vae(x)
            loss = ((x - x_hat)**2).sum() + beta * vae.encoder.kl
            loss.backward()
            opt.step()
            vae.decoder.sparse_layer[0].constrain_weights()
            train_loss_e += loss.detach().cpu().item()

        # start updating our SWA weights
        if epoch >= swa_start:
            swa_model.update_parameters(vae)
            swa_scheduler.step()

        train_losses.append(train_loss_e / (len(data) * data.batch_size))

        # validation
        vae.eval()
        valid_loss_e = 0.
        with torch.no_grad():
            for x in val_data:
                x = x.to(device)
                x_hat = vae(x)
                loss = ((x - x_hat)**2).sum() + beta * vae.encoder.kl
                valid_loss_e += loss.detach().cpu().item()
        valid_losses.append(valid_loss_e / (len(val_data) * val_data.batch_size))

        if epoch % 10 == 0:
            print(f"Epoch {epoch}: train_loss={train_losses[-1]:.4f}, valid_loss={valid_losses[-1]:.4f}")

    # recompute batch‐norm statistics for the SWA model (if you had any)
    update_bn(data, swa_model, device=device)

    return swa_model, train_losses, valid_losses

### 3. Bayes model + merged mask

In [ ]:
class MergedBayesDecoder(nn.Module):
    """
    Define VEGA's decoder (sparse, one-layer, linear, positive)
    """
    def __init__(self, mask, soft_penalty):
        super(MergedBayesDecoder, self).__init__()

        self.sparse_layer = nn.Sequential(MergedBayesianSparseLayer(mask, soft_penalty)) # we define the architecture of the decoder below with the class "SparseLayer"

    def forward(self, x):
        z = self.sparse_layer(x.to(device))
        return(z)

    def kl_divergence(self):
        return self.sparse_layer[0].kl_loss()

# define a class SparseLayer, that specifies the decoder architecture (sparse connections based on the mask)
class MergedBayesianSparseLayer(BayesianSparseLayer):
    def __init__(self, mask, soft_penalty):
        """
        Extended torch.nn module which mask connection
        """
        super(MergedBayesianSparseLayer, self).__init__(mask, soft_penalty)

        self.activator_mask = (self.mask == 1).float()
        self.inhibitor_mask = (self.mask == -1).float()
        self.zero_mask = (self.mask == 0).float()

    def reset_parameters(self):
        # Initialization method of Adv-BNN
        stdv = 1. / math.sqrt(self.weight_mu.size(1))
        self.weight_mu.data.uniform_(-stdv, stdv)
        self.weight_log_sigma.data.fill_(self.prior_log_sigma)
        self.bias_mu.data.uniform_(-stdv, stdv)
        self.bias_log_sigma.data.fill_(self.prior_log_sigma)

    def constrain_weights(self):
        with torch.no_grad():
            # Clamp activator weights to be >= 0
            device = self.weight_mu.device

            a_mask = self.activator_mask.to(device)
            i_mask = self.inhibitor_mask.to(device)
            z_mask = self.zero_mask.to(device)

            self.weight_mu.data = self.weight_mu.data * (1 - a_mask) + \
                               self.weight_mu.data.clamp(min=0) * a_mask

            # Clamp inhibitor weights to be <= 0
            self.weight_mu.data = self.weight_mu.data * (1 - i_mask) + \
                               self.weight_mu.data.clamp(max=0) * i_mask

            # Set zero-regulation weights to exactly zero
            self.weight_mu.data = self.weight_mu.data * (1 - z_mask)


class MergedBayesVEGA(nn.Module):
    def __init__(self, latent_dims, input_dims, mask, dropout = 0.3, z_dropout = 0.3, soft_penalty = 0.0001):
        super(MergedBayesVEGA, self).__init__()
        self.encoder = Encoder(latent_dims, input_dims, dropout, z_dropout) # we use the same encoder as before (two-layer, fully connected, non-linear)
        self.decoder = MergedBayesDecoder(mask, soft_penalty)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [ ]:
### training loop
def trainMergedVEGA_with_Bayes(vae, data, val_data, epochs=100, beta_en = 0.0001, beta_de = 0.0001, learning_rate = 0.0001):
    opt = torch.optim.Adam(vae.parameters(), lr = learning_rate, weight_decay = 5e-4)
    train_losses = []
    valid_losses = []
    for epoch in range(epochs):
        train_loss_e = 0
        valid_loss_e = 0
        vae.train() #train mode

        for x in data:
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = vae(x)
            kl_encoder = vae.encoder.kl
            kl_decoder = vae.decoder.kl_divergence()

            loss = ((x - x_hat)**2).sum() + beta_en* kl_encoder + beta_de*kl_decoder
            loss.backward()
            opt.step()
            train_loss_e += loss.to('cpu').detach().numpy()
            vae.decoder.sparse_layer[0].constrain_weights()
        train_losses.append(train_loss_e/(len(data)*128))

        #### Here you should add the validation loop
        vae.eval()

        for x in val_data:
            x = x.to(device)
            x_hat = vae(x)
            kl_encoder = vae.encoder.kl
            kl_decoder = vae.decoder.kl_divergence()
            loss = ((x - x_hat)**2).sum() + beta_en* kl_encoder + beta_de*kl_decoder
            valid_loss_e += loss.to('cpu').detach().numpy()
        valid_losses.append(valid_loss_e/(len(val_data)*128))

        if epoch % 10 == 0:
            print("epoch: ", epoch, " train_loss: ", train_loss_e/(len(data)*128), "  valid_loss: ", valid_loss_e/(len(val_data)*128))

    return vae, train_losses, valid_losses


### Run merged models

In [ ]:
def run_merged_vega_model(model_type, train_data, valid_data, cond, activator_gmt, inhibitor_gmt, N):
    """
    model_type: "vega", "swa", "bayes"
    cond is "stimulated" or "control"
    activator_gmt is "activator.gmt" or "STAT_activator.gmt"
    inhibitor_gmt is "inhibitor.gmt" or "STAT_inhibitor.gmt"
    """

    # create mask

    train_data, mask = create_merged_mask(train_data, activator_gmt, inhibitor_gmt, add_nodes=1)
    train_data = train_data[train_data.obs["condition"] == cond]
    valid_data = valid_data[valid_data.obs["condition"] == cond]

    # set up dataloader
    trainX = torch.utils.data.DataLoader(train_data.X.toarray(), batch_size=128, shuffle=True)
    validX = torch.utils.data.DataLoader(valid_data.X.toarray(), batch_size=128, shuffle=True)

    all_weights = []

    match model_type:
        case "vega":
            for _ in range(N):
                vega = MergedVEGA(latent_dims= mask.shape[1], input_dims = mask.shape[0], mask = mask.T, dropout = 0.3, z_dropout = 0.3, soft_penalty=0.1).to(device)
                vega, train_losses, valid_losses = trainMergedVEGA_with_valid(vega, trainX, validX, epochs = 50, beta = 0.0001)
                weight = get_weight(vega)
                all_weights.append(weight)
                plot_loss_curve(train_losses, valid_losses)

        case "swa":
            for _ in range(N):
                vega = MergedVEGA(latent_dims=mask.shape[1],
                        input_dims=mask.shape[0],
                        mask=mask.T,
                        dropout=0.3,
                        z_dropout=0.3, soft_penalty=0.1).to(device)

                swa_model, train_losses, valid_losses = trainMergedVEGA_with_swa(
                    vega, trainX, validX,
                    epochs=50, beta=0.0001,
                    learning_rate=0.001,
                    swa_start=80,      # you can tune this
                    swa_lr=0.0005        # and this
                )

                W_swa = get_weight_swa(swa_model)
                all_weights.append(W_swa)
                plot_loss_curve(train_losses, valid_losses)

        case "bayes":
            weight_uncertainties = []
            for _ in range(N):
                vega = MergedBayesVEGA(latent_dims= mask.shape[1], input_dims = mask.shape[0], mask = mask.T, dropout = 0.3, soft_penalty = 0.0001).to(device)
                vega, train_losses, valid_losses = trainMergedVEGA_with_Bayes(vega, trainX, validX, epochs = 60, beta_en = 0.0001, beta_de=0.0001)
                weight = get_weight_bayes(vega)
                uncertainty = get_weight_uncertainties_bayes(vega)
                all_weights.append(weight)
                weight_uncertainties.append(uncertainty)
                plot_loss_curve(train_losses, valid_losses)

            weight_uncertainties = np.stack(weight_uncertainties)
            mean_weight_uncertainties = weight_uncertainties.mean(axis=0)
            mean_weight_uncertainties = pd.DataFrame(mean_weight_uncertainties, index = train_data.var.index.tolist(), columns=train_data.uns['_vega']['gmv_names'])
            mean_weight_uncertainties.to_csv(f"{cond}_merged_{model_type}_weight_uncertainty_mean.csv", index=True)


    stacked = np.stack(all_weights)
    mean_weight = stacked.mean(axis=0)
    std_weight = stacked.std(axis=0)

    mean_weight = pd.DataFrame(mean_weight, index = train_data.var.index.tolist(), columns=train_data.uns['_vega']['gmv_names'])
    std_weight = pd.DataFrame(std_weight, index = train_data.var.index.tolist(), columns=train_data.uns['_vega']['gmv_names'])

    mean_weight.to_csv(f"{cond}_merged_{model_type}_weight_mean.csv", index=True)
    std_weight.to_csv(f"{cond}_merged_{model_type}_weight_std.csv", index=True)

    return vega, mean_weight, std_weight

# Plotting function

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## CHANGE these values to generate different plots
TF = "STAT1"
TF_type = "activator"
model = "vega"

# 1) Load means (you already have these)
mean_weight_stim = pd.read_csv(f"stimulated_{TF_type}_{model}_weight_mean.csv", index_col=0)
mean_weight_ctrl = pd.read_csv(f"control_{TF_type}_{model}_weight_mean.csv", index_col=0)

stim = mean_weight_stim[TF].rename("stim")
ctrl = mean_weight_ctrl[TF].rename("ctrl")

# 2) Load the std‐weight CSVs in exactly the same shape
std_weight_stim  = pd.read_csv(f"stimulated_{TF_type}_{model}_weight_std.csv", index_col=0)
std_weight_ctrl  = pd.read_csv(f"control_{TF_type}_{model}_weight_std.csv",  index_col=0)

std_stim = std_weight_stim[TF].rename("std_stim")
std_ctrl = std_weight_ctrl[TF].rename("std_ctrl")

# 3) Merge everything on the gene index, dropping any missing
merged = pd.concat([ctrl, stim, std_ctrl, std_stim], axis=1).dropna()

# 4) Compute a single uncertainty score per point
#    Here we take the mean of the two std’s, but you could also use max(), sum(), etc.
merged["uncertainty"] = merged[["std_ctrl","std_stim"]].mean(axis=1)

# 5) Scatter with color = uncertainty
plt.figure(figsize=(6,6))
sc = plt.scatter(merged["ctrl"], merged["stim"],
                 c=merged["uncertainty"],
                 cmap="viridis",
                 s=20,             # marker size
                 edgecolor="k",    # thin black outline
                 alpha=0.8)

# 6) Diagonal & axes
plt.plot([merged["ctrl"].min(), merged["ctrl"].max()],
         [merged["ctrl"].min(), merged["ctrl"].max()],
         color='grey', linestyle='--')
plt.xlabel("Control mean weight")
plt.ylabel("Stimulated mean weight")
plt.title(f"{TF} weight: control vs. stimulated")

# 7) Colorbar
cbar = plt.colorbar(sc, label="Mean STD (uncertainty)")

# 8) Optional: annotate outliers as before
merged["dist"] = np.abs(merged["stim"] - merged["ctrl"]) / np.sqrt(2)
top = merged.nlargest(20, "dist")
for gene, row in top.iterrows():
    plt.annotate(gene, (row["ctrl"], row["stim"]), fontsize=8)

plt.tight_layout()
plt.show()